In [1]:
import pandas as pd

plans = pd.read_csv('plans.csv', usecols=['metal_level', 'rate', 'rate_area'])
zips_and_rate_areas = pd.read_csv(
    'zips.csv', 
    usecols=['zipcode', 'rate_area'], 
    dtype={'zipcode': str}
    )
target_zips = pd.read_csv('slcsp.csv', usecols=['zipcode'], dtype={'zipcode': str})

silver_plans = plans[plans['metal_level'] == 'Silver']

In [2]:
plan_area_counts = silver_plans['rate_area'].value_counts()
has_multiple_plans = plan_area_counts.values > 1
multi_plan_areas = plan_area_counts.index[has_multiple_plans]
multi_plan_rates = silver_plans[silver_plans['rate_area'].isin(multi_plan_areas)]

In [3]:
zips_and_rates = pd.merge(
    zips_and_rate_areas, 
    multi_plan_rates,  
    on='rate_area'
    )

In [4]:
len(zips_and_rates)

24500774

In [5]:
zips_and_rates.head()

,zipcode,rate_area,metal_level,rate
0,36749,11,Silver,310.12
1,36749,11,Silver,348.73
2,36749,11,Silver,334.38
3,36749,11,Silver,330.08
4,36749,11,Silver,249.72


In [6]:
# Create df where each row has a unique (zipcode, rate_area) combination
unique_zip_rate_areas = zips_and_rates.drop_duplicates(
    subset=['zipcode', 'rate_area']
    )

In [7]:
# Zipcode frequency counts give # of rate areas a particular zipcode encompasses
num_rate_areas_per_zip = unique_zip_rate_areas['zipcode'].value_counts()

In [8]:
# Identify zipcodes that map to a single rate area, and extract the
# associated plan data
is_unambiguous = num_rate_areas_per_zip.values == 1
unambiguous_zips = num_rate_areas_per_zip[is_unambiguous].index
unambiguous_plans = zips_and_rates[zips_and_rates['zipcode'].isin(unambiguous_zips)]

In [39]:
# Only consider zipcodes with more than one silver plan
have_multiple = (unambiguous_plans['zipcode'].value_counts() > 1)

In [50]:
len(unique_zip_and_rate_area['rate'].value_counts().values > 1)

67

In [9]:
# set => unique rates & sorted => we can safely take second element
# as slcsp without having to separately check for repeated lowest rate)
slcsp_by_zip = unambiguous_plans.groupby('zipcode').agg(
    {'rate':lambda x: sorted(set(x))[1]}
    ).reset_index() # convert multi-index df from groupby to standard indexing 

In [19]:
target_slcsps = slcsp_by_zip[slcsp_by_zip['zipcode'].isin(target_zips.values.flatten())]

In [24]:
results = pd.merge(target_zips, target_slcsps, how='left', on='zipcode').fillna('').set_index('zipcode')

In [26]:
results.to_csv('test.csv')

In [30]:
results.to_csv(None)

'zipcode,rate\n64148,195.66\n67118,165.5\n40813,237.91\n18229,195.66\n51012,195.66\n79168,184.97\n54923,\n67651,166.13\n49448,221.63\n27702,235.65\n47387,235.65\n50014,166.13\n33608,268.48\n06239,237.91\n54919,235.65\n46706,\n14846,\n48872,\n43343,\n77052,235.92\n07734,166.13\n95327,166.13\n12961,209.35\n26716,214.34\n48435,\n53181,214.34\n52654,184.97\n58703,166.13\n91945,226.84\n52146,209.35\n56097,\n21777,166.13\n42330,184.97\n38849,184.97\n77586,235.92\n39745,165.5\n03299,166.13\n63359,\n60094,166.13\n15935,184.97\n39845,196.64\n48418,\n28411,196.64\n37333,195.66\n75939,234.5\n07184,166.13\n86313,184.97\n61232,165.5\n20047,166.13\n47452,\n31551,165.5\n'

In [12]:
# merge dfs, using left join to preserve original ordering
#test = pd.merge(slcsp_df, slcsp_by_zip, how='left', on='zipcode')

In [44]:
zips_and_rate_areas[zips_and_rate_areas['zipcode']=='54923']

,zipcode,rate_area
50257,54923,15
51100,54923,11
51117,54923,11


In [37]:
unambiguous_plans.head()

,zipcode,rate_area,metal_level,rate
710,36003,11,Silver,310.12
711,36003,11,Silver,348.73
712,36003,11,Silver,334.38
713,36003,11,Silver,330.08
714,36003,11,Silver,249.72


In [45]:
len(results)

51